# Importing Libraries

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import seed
from numpy.random import randn
import numpy as np
from scipy import stats
from math import sqrt
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from scipy.stats import sem
from scipy.stats import t
from termcolor import colored

# Population

In [2]:
ovaries_ddct = pd.read_excel('ovaries_exp_ddct.xlsx')
ovaries_ddct.describe()

,Unnamed: 0,RPL32,VGR,VG,Day,DCT_VG,DCT_VGR,DDCT_VG,DDCT_VGR,EXP_DDCT_VG,EXP_DDCT_VGR
count,78.000000,78.000000,78.00000,77.000000,78.000000,77.000000,78.000000,77.000000,78.000000,77.000000,78.000000
mean,38.500000,18.600436,26.19741,29.356403,4.000000,10.764442,7.596974,6.448442,2.161974,0.047489,0.321053
std,22.660538,0.843461,1.67892,2.980935,3.006486,2.630613,1.352549,2.630613,1.352549,0.158958,0.247151
min,0.000000,16.984000,23.14400,23.278000,0.000000,4.316000,5.435000,0.000000,0.000000,0.000017,0.022593
25%,19.250000,17.979000,25.02800,28.038000,1.000000,9.529000,6.315250,5.213000,0.880250,0.006551,0.125212
50%,38.500000,18.638000,26.13050,28.683000,4.000000,10.359000,7.536500,6.043000,2.101500,0.015166,0.233044
75%,57.750000,19.276250,27.14975,30.028000,8.000000,11.570000,8.433250,7.254000,2.998250,0.026961,0.543289
max,77.000000,20.269000,31.17200,40.000000,8.000000,20.160000,10.903000,15.844000,5.468000,1.000000,1.000000


Each distribution is a total of eggs layed during 10 days for each female individual

In [3]:
carcass_ddct = pd.read_excel('carcass_exp_ddct.xlsx')
carcass_ddct.describe()

,Unnamed: 0,RPL32,VGR,VG,Day,DCT_VG,DCT_VGR,DDCT_VG,DDCT_VGR,EXP_DDCT_VG,EXP_DDCT_VGR
count,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000
mean,38.500000,19.876526,29.664344,25.870927,4.000000,5.994400,9.787818,3.098717,4.578047,0.266947,0.095924
std,22.660538,1.014678,2.232465,3.226914,3.006486,3.124633,2.002313,3.124633,2.002313,0.229351,0.159893
min,0.000000,18.463029,24.188826,21.891192,0.000000,2.895684,5.209771,0.000000,0.000000,0.000064,0.000806
25%,19.250000,19.096639,27.913504,23.845645,1.000000,4.276703,8.530594,1.381020,3.320824,0.101291,0.017128
50%,38.500000,19.626455,29.592190,25.584168,4.000000,5.257014,9.599196,2.361330,4.389425,0.194612,0.047715
75%,57.750000,20.647191,31.281618,26.415266,8.000000,6.200101,11.078975,3.304418,5.869204,0.383951,0.100092
max,77.000000,22.333588,34.734440,35.986200,8.000000,16.824016,15.486190,13.928332,10.276420,1.000000,1.000000


## Sanity Test

In [4]:
def interpret_ttest(a, b, alpha=0.05):
    t_stat, df, cv, p = independent_ttest(a, b, alpha)
    print('t=%.3f, df=%d, cv=%.5f, p=%.8f' % (t_stat, df, cv, p))
#     # interpret via critical value
#     if abs(t_stat) <= cv:
#         print('Accept null hypothesis (Same distributions).')
#     else:
#         print('Reject the null hypothesis (Different distributions).')
    # interpret via p-value
    if p > alpha:
        print(colored('Accept null hypothesis (Same distributions).', 'green'))
    else:
        print(colored('Reject the null hypothesis (Different distributions).', 'red'))

In [5]:
def interpret_kwtest(a, b, alpha = 0.05):
    stat, p = stats.kruskal(a, b)
    print('Statistics=%.3f, p=%.8f' % (stat, p))
    # interpret

    if p > alpha:
        print(colored('Same distributions (fail to reject H0)', 'green'))
    else:
        print(colored('Different distributions (reject H0)', 'red'))

In [6]:
from scipy.stats import shapiro
# normality test
# interpret results
def interpret_normaltest(data, alpha=0.05):
    stat, p = shapiro(data)
    print('Statistics=%.3f, p=%.3f' % (stat, p))

    if p > alpha:
        print(colored('Sample looks Gaussian (fail to reject H0)', 'green'))
    else:
        print(colored('Sample does not look Gaussian (reject H0)', 'red'))



## Comparison Scenarios

In [7]:
groups = ovaries_ddct['Group'].unique()
groups

array(['A', 'P', 'B', 'C', 'D'], dtype=object)

In [8]:
carcass_vg  = {g: carcass_ddct[carcass_ddct['Group'] == g]['EXP_DDCT_VG'].values for g in groups}
carcass_vgr  = {g: carcass_ddct[carcass_ddct['Group'] == g]['EXP_DDCT_VGR'].values for g in groups}

In [9]:
ovaries_vg  = {g: ovaries_ddct[ovaries_ddct['Group'] == g]['EXP_DDCT_VG'].values for g in groups}
ovaries_vgr  = {g: ovaries_ddct[ovaries_ddct['Group'] == g]['EXP_DDCT_VGR'].values for g in groups}

In [10]:
scenarios_vg_comp_groups = {f'{g}-Carcass-Ovaries': (ovaries_vg[g],carcass_vg[g])   for g in groups}
scenarios_vgr_comp_groups = {f'{g}-Carcass-Ovaries': (ovaries_vgr[g],carcass_vgr[g])   for g in groups}

### Statistical Tests VG


#### Inter Group VG

In [11]:
for k, v in scenarios_vg_comp_groups.items():
    a, b = v
    print(f'Scenario {k}')
    interpret_kwtest(a,b)
    print('\n\n')

Scenario A-Carcass-Ovaries
Statistics=26.270, p=0.00000030
Different distributions (reject H0)



Scenario P-Carcass-Ovaries
Statistics=nan, p=nan
Different distributions (reject H0)



Scenario B-Carcass-Ovaries
Statistics=26.274, p=0.00000030
Different distributions (reject H0)



Scenario C-Carcass-Ovaries
Statistics=12.004, p=0.00053092
Different distributions (reject H0)



Scenario D-Carcass-Ovaries
Statistics=17.441, p=0.00002963
Different distributions (reject H0)





### Statistical Tests VGR

#### Inter Group VGR

In [12]:
for k, v in scenarios_vgr_comp_groups.items():
    a, b = v
    print(f'Scenario {k}')
    interpret_kwtest(a,b)
    print('\n\n')

Scenario A-Carcass-Ovaries
Statistics=9.811, p=0.00173489
Different distributions (reject H0)



Scenario P-Carcass-Ovaries
Statistics=8.308, p=0.00394775
Different distributions (reject H0)



Scenario B-Carcass-Ovaries
Statistics=18.243, p=0.00001944
Different distributions (reject H0)



Scenario C-Carcass-Ovaries
Statistics=6.247, p=0.01243865
Different distributions (reject H0)



Scenario D-Carcass-Ovaries
Statistics=18.788, p=0.00001461
Different distributions (reject H0)



